In [59]:
import os
import re
from dotenv import load_dotenv
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
path="./data_dump/WhatsApp Chat with Team Lending.txt"
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhiv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000027DB34F0650> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000027DB358D1D0> root_client=<openai.OpenAI object at 0x0000027DB34BAC90> root_async_client=<openai.AsyncOpenAI object at 0x0000027DA5749650> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [61]:
with open(path, "r", encoding="utf-8") as file:
    lines = file.readlines()
    for line in lines[:10]:
        print(line.strip())


02/09/24, 10:54 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.
09/08/24, 10:55 - ~ Munwar Sk created group "Team Lending"
02/09/24, 10:54 - Junaid Dbs added you
02/09/24, 11:19 - ~ Eeshwar Ankathi added ~ puja and ~ Rhutuja Tanpure
02/09/24, 17:19 - Navya Dbs: <Media omitted>
02/09/24, 17:20 - +91 90032 71032: <Media omitted>
04/09/24, 11:00 - Shailender Rawat: <Media omitted>
04/09/24, 11:00 - Shailender Rawat: Bade Miyaan Chote Miyaan
08/09/24, 11:44 - +91 93928 08544: Hey everyone!



In [62]:
def parse_whatsapp_dump(file_path):
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}) - ([^:]+): (.+)'
    messages = []
    
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            match = re.match(pattern, line)
            if match:
                timestamp, sender, content = match.groups()
                if "<Media omitted>" not in content:
                    content = re.sub(r'[^A-Za-z0-9\s]', '', content)
                    messages.append({"timestamp": timestamp, "sender": sender, "content": content})
    
    return pd.DataFrame(messages)
extracted_data=parse_whatsapp_dump(path)
extracted_data.to_csv("extracted_data_lending_chat.csv")

In [63]:

def remove_stop_words(words):
    words=words.split(" ")
    sen_list=[word for word in words if word.lower() not in stop_words]
    return " ".join(sen_list)


data=extracted_data.copy()

data["content"]=data["content"].apply(remove_stop_words)

data["date"]=pd.to_datetime(data["timestamp"], format='%d/%m/%y, %H:%M')
data["date"]=data["date"].dt.date



In [64]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = PromptTemplate(
        input_variables=["message"],
    template=(
 """
You are an assistant that detects birthday messages from WhatsApp chat logs.
Your task is to determine if a message contains a birthday wish and identify the person whose birthday it is.

- If the message contains a birthday wish, return `True` for `is_it_birthday` and the name of the person whose birthday it is.
- If the name of the person is not explicitly mentioned in the message, look for any phone number that may be mentioned instead. If no name and no phone number are found, return `None` for `whos_birthday`.
- Since the messages are from a group chat, the birthday wish may be directed to one person. The name might be directly mentioned, or a phone number may be shared instead. If a name is mentioned, it is the name of the person whose birthday it is.
- If no birthday wish is detected, return `False` for `is_it_birthday` and `None` for `whos_birthday`.
- Only return the results without further analysis or explanation. Do not output reasoning. Format the results as a tuple: (is_it_birthday, whos_birthday).
- **Only one result should be returned, even if multiple people or phone numbers are mentioned.**

Message: {message}

Example 1:
Message: "[Happy Birthday John! Hope you have an amazing day., Happy Birthday john, wish you a happy birthday John abraham]"
Output: True, John

Example 2:
Message: "[Happy birthday 135445134684, many more happy returns of the day abhishek, abhishek happy birthday]"
Output: True, abhishek

Example 3:
Message: "Hey, what's up?"
Output: False, None

Example 4:
Message: "It's my birthday today, reach out to me at 9876543210!"
Output: True, 9876543210

Example 5:
Message: "Happy Birthday to the best friend ever! Call me at 3216549870."
Output: True, 3216549870


"""
    ),
)



chain= {"message": RunnablePassthrough() } | prompt | llm | StrOutputParser()


In [73]:
df=pd.DataFrame(columns=["date", "name", "group"])
iter=0
def split_and_check(input_str):
    try:
        # Split the string by comma and check the length
        result = input_str.split(",")
        
        # If the length is not 2, return a message indicating the issue
        if len(result) != 2:
            print(f"Warning: Result does not have length 2. Result: {result}")
            result=["False", None]
        return result
    
    except Exception as e:
        print(input_str)
        print(f"Error encountered: {e}")
        return ["False", None]
for i,j in data.groupby("date"):
    
    j = j.drop_duplicates(subset='content', keep='first')
    messages=j["content"].to_list()
    output=chain.invoke(messages)
    print(output)
    output=split_and_check(output)
    
    print(i,output[1],"lending team")
    if output[0]=="True":
        
        df.loc[iter] = [i,output[1],"lending team"]
        iter=iter+1
print(df)

False, None
2024-09-04  None lending team
False, None
2024-09-08  None lending team
True, Sudeeksha
2024-09-09  Sudeeksha lending team
False, None
2024-09-11  None lending team
False, None
2024-09-20  None lending team
False, None
2024-09-25  None lending team
True, Manish
2024-09-26  Manish lending team
False, None
2024-10-01  None lending team
False, None
2024-10-02  None lending team
False, None
2024-10-03  None lending team
True, 919849943444
2024-10-04  919849943444 lending team
False, None
2024-10-08  None lending team
False, None
2024-10-23  None lending team
False, None
2024-10-25  None lending team
False, None
2024-10-26  None lending team
False, None
2024-10-29  None lending team
True, Pratap
2024-11-11  Pratap lending team
False, None
2024-11-15  None lending team
False, None
2024-11-20  None lending team
False, None
2024-11-23  None lending team
False, None
2024-11-24  None lending team
False, None
2024-11-25  None lending team
False, None
2024-11-26  None lending team
Fals